In [ ]:
dataset_name = 'CA'

os.makedirs(f"results/{dataset_name}", exist_ok=True)

# Load dataset and parameters
df, numeric_features, threshold = load_dataset(dataset_name)
print(df.head(1))

# Create graph
G = create_graph_from_dataframe(df, numeric_features, distance_threshold=threshold)

# Run experiments
results = run_experiments(df, G, dataset_name)
print(results)

In [ ]:
import os
import argparse
from utils_embeddings import load_dataset, create_graph_from_dataframe, run_experiments

import pandas as pd
import numpy as np
import networkx as nx
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_percentage_error, mean_squared_error
import matplotlib.pyplot as plt
import os
from scipy.spatial import cKDTree

# import embedding functions
from embeddings_utils import *
from utils_embeddings import *

In [ ]:
# print(df.shape)
results = []

# -------------------
# Baseline (raw features)
# -------------------
X_base = df.drop(['price', 'id'], axis=1)
y = df['price']
X_train_base, X_test_base, y_train, y_test = train_test_split(X_base, y, test_size=0.1, random_state=42)
X_train_base.shape, X_test_base.shape, y_train.shape, y_test.shape

In [ ]:
for model_name, model in [
    ("GradientBoosting", GradientBoostingRegressor(random_state=42)),
    ("LinearRegression", LinearRegression()),
    ("RandomForest", RandomForestRegressor(random_state=42))
]:
    metrics = fit_and_evaluate(model, X_train_base, y_train, X_test_base, y_test, verbose=False)
    results.append([f"{model_name} (Raw)", *metrics])

In [ ]:
results

In [ ]:
%%time
%run main_hpp.py --dataset CA
# python main_hpp.py --dataset MHD
